<a href="https://colab.research.google.com/github/Srikanth9142/ML-works/blob/master/pkl_file_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
####This notebook is for running after saving the data in pickle files 

##Mounting google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

##Unzipping folder

In [0]:
!unzip -q "/content/drive/My Drive/dataset_updated_20.zip"
%ls


## Installed && imported modules

In [0]:
pip install python_speech_features


  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5889 sha256=715e75ff54b6dd08574bb638e0f17e10fdec5584a69962500d05ea3509ff96f7
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
from python_speech_features import mfcc,delta,logfbank 
from tqdm import tnrange
import numpy
import scipy.io.wavfile
import numpy as np
import wave
import contextlib
from scipy.fftpack import dct
import scipy.io.wavfile as wav
import pickle

##Loading the pickle files into arrays for training


In [0]:
##add pickle file path(x-train)
with("#",as "rb")as f:
  x_train = pickle.load(f)


In [0]:
##add pickle file path(x-test)
with("#",as "rb")as f:
  x_test = pickle.load(f)


In [0]:
##add pickle file path(y-train)
with("#",as "rb")as f:
  y_train= pickle.load(f)


In [0]:
##add pickle file path(y_test)
with("#",as "rb")as f:
  y_test= pickle.load(f)


##Imports needed for model

In [0]:
from keras import applications
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization,AveragePooling2D
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D,Dropout,Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.models import load_model


##Function for Model building

In [0]:
##create model function 
def create_model():
  base_model=applications.resnet50.ResNet50(weights=None,include_top=False,input_shape = (40,40,1))
  x = base_model.output
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = AveragePooling2D(pool_size=2)(x)
  z1 = Flatten()(x)
  outputs = Dense(11,activation='softmax',kernel_initializer='he_normal')(z1)
  rnet=Model(inputs=base_model.input,outputs=outputs)
  model = Sequential()
  model.add(Convolution2D(1,(3,3),padding='same',activation = 'relu',input_shape = (398,40,1)))
  model.add(MaxPooling2D(pool_size=(2, 1)))
  model.add(MaxPooling2D(pool_size=(2, 1)))
  model.add(MaxPooling2D(pool_size=(2, 1)))
  model.add(Convolution2D(1, 2, 1, activation='relu',use_bias=False))
  model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
  model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
  model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
  model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
  model.add(rnet)
  adam=Adam(lr=0.0001)
  model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
  model.summary()
  return model



##Functions for prediction


In [0]:
def prediction(targ):
    accents = ['bengali','dogri','gujarathi','hindi','malayalam','manipuri','marathi','odisha','tamil','telugu','urdu']
    np.set_printoptions(formatter={'float_kind':'{:.1f}'.format})
    #np.set_printoptions(formatter={'float_kind':'{:.1f}'.format})
    l=[]
    out_len = len(targ)
    in_len = len(targ[0])
    for i in range(in_len):
        sumi = 0
        for j in range(out_len):
            sumi+=targ[j][i]
        l.append(sumi)
    print(l)
    maximum = -1
    max_ind = -1
    for i in range(len(l)):
        if(l[i] > maximum):
            maximum = l[i]
            max_ind = i+1
    # print(maximum, max_ind)
    print("Accent: ",accents[max_ind-1])
    for i in l:
        if i!=0:
            per = (i/float(out_len))*100
            if(per>1):
                print("{1:.2f}% -> {0}".format(accents[l.index(i)],per))

In [0]:
## This is for testing 
def Pre_final_data(folder,files):
    index=0
    
    mfcc=[]
    target=[]
    mfcc_len=length_audio(folder,files)
    count=0
    for z in range(0,mfcc_len,2):
      count=count+1
    data_dic = np.zeros((count,398,40))
    for i in range(0,mfcc_len,2):
        
        mfcc = create_mfcc(folder,files,i)
        for j in range(len(mfcc)):
              for k in range(len(mfcc[j])):
                data_dic[index][j][k] = mfcc[j][k]
        index += 1
        #target.append(mfcc)
      #target=np.asarray()
    
    data_dic_reshaped = data_dic.reshape(data_dic.shape[0],398,40,1)
    model = create_model()
    #model.load_weights('drive/My Drive/Language3.h5')
    model.load_weights('drive/My Drive/Language3.h5')
    targ=model.predict(data_dic_reshaped)    
    print(targ)
    print(prediction(targ))
    #print(data_dic_reshaped)

##Functions need for testing data

In [0]:
# variables used in MFCC
pre_emphasis = 0.97
frame_size = 0.025
frame_stride = 0.01
NFFT = 512
nfilt = 40
num_ceps = 12
cep_lifter =22

In [0]:
# The below function is to find the length of the audio with given input foldername and filename
def length_audio(folder_name,file_name):
  file_path='/content/'+str(folder_name)+"/"+str(file_name)
  sample_rate, signal = scipy.io.wavfile.read(file_path)
  with contextlib.closing(wave.open(file_path,'r')) as f:
      frames = f.getnframes()
      rate = f.getframerate()
      duration = frames / float(rate)
  return int(duration)

In [0]:
def create_mfcc(folder_name,file_name,start_point):
    file_path ='/content/'+str(folder_name)+"/"+str(file_name)
    sample_rate, signal = scipy.io.wavfile.read(file_path)
    signal = signal[start_point:int(start_point+2 * sample_rate)]   #framing to 2 seconds
    emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1]) #pre-emphasis
    frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
    pad_signal_length = num_frames * frame_step + frame_length
    z = numpy.zeros((pad_signal_length - signal_length))
    pad_signal = numpy.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal
    indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(numpy.int32, copy=False)]
    #After slicing the signal into frames, we apply a window function such as the Hamming window
    frames *= numpy.hamming(frame_length)  #hamming window
    mag_frames = numpy.absolute(numpy.fft.rfft(frames, NFFT))  # Magnitude of the FFT
    pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum
    low_freq_mel = 0
    high_freq_mel = (2595 * numpy.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
    mel_points = numpy.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
    hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
    bin = numpy.floor((NFFT + 1) * hz_points / sample_rate)

    fbank = numpy.zeros((nfilt, int(numpy.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])   # left
        f_m = int(bin[m])             # center
        f_m_plus = int(bin[m + 1])    # right

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])

    filter_banks = numpy.dot(pow_frames, fbank.T)
    filter_banks = numpy.where(filter_banks == 0, numpy.finfo(float).eps, filter_banks)  # Numerical Stability
    filter_banks = 20 * numpy.log10(filter_banks)  # dB
    mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # Keep 2-13
    (nframes, ncoeff) = mfcc.shape
    n = numpy.arange(ncoeff)
    lift = 1 + (cep_lifter / 2) * numpy.sin(numpy.pi * n / cep_lifter)
    mfcc *= lift  #*
    
    filter_banks -= (numpy.mean(filter_banks, axis=0) + 1e-8)
    mfcc -= (numpy.mean(mfcc, axis=0) + 1e-8)
    
    return filter_banks

##Fit the model


In [0]:
model = create_model()
model.fit(x_train,y_train,epochs = 25,batch_size=64)
